# Carga Silver - Stage Cotação Histórica

Este notebook realiza a carga na silver cotação histórica (stg_cotacao_historica) a partir dos dados da tabela bronze cotação histórica (raw_cotahist).

## Imports

In [1]:
from spark_config import init_spark
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType, IntegerType

## Start Spark Session

In [2]:
spark = init_spark("Leitura e transformação delta table")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Variables

In [3]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"

# Define o caminho da tabela Delta Source
delta_path_cotahist = f"{base_bronze_path}/raw_cotahist"

# Define o caminho para salvar a tabela Delta transformada
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"

## Read Table

In [4]:
# Lendo a tabela Delta no DataFrame do PySpark
df_bronze = spark.read.format("delta").load(delta_path_cotahist)

print(f"[SUCESSO] Leitura da tabela Delta em: {delta_path_cotahist}")
print(f"Total de registros: {df_bronze.count()}")

[SUCESSO] Leitura da tabela Delta em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cotahist
Total de registros: 3094053


## Transformation

In [5]:
# Transformações e seleção de colunas
df_silver = df_bronze.select(
    # Campos String
    F.col("tipo_registro").cast("string").alias("tp_registro"),
    F.to_date(F.col("data_pregao"), "yyyyMMdd").alias("dt_pregao"),
    F.col("codigo_bdi").cast("string").alias("cd_bdi"),
    F.trim(F.col("codigo_negociacao")).alias("cd_negociacao"),
    F.col("tipo_mercado").cast("string").alias("tp_mercado"),
    F.trim(F.col("nome_resumido_empresa")).alias("nm_empresa"),
    F.trim(F.col("especificacao_papel")).alias("ds_especificacao_papel"),
    F.col("prazo_dias_mercado").cast("string").alias("nr_prazo_dias_mercado"),
    F.col("moeda_referencia").cast("string").alias("cd_moeda_referencia"),
    
    # Campos Decimais com precisão 11,2
    (F.col("preco_abertura_papel") / 100).cast(DecimalType(11,2)).alias("vl_abertura"),
    (F.col("preco_maximo_papel") / 100).cast(DecimalType(11,2)).alias("vl_maximo"),
    (F.col("preco_minimo_papel") / 100).cast(DecimalType(11,2)).alias("vl_minimo"),
    (F.col("preco_medio_papel") / 100).cast(DecimalType(11,2)).alias("vl_medio"),
    (F.col("preco_ultimo_negocio") / 100).cast(DecimalType(11,2)).alias("vl_ultimo_negocio"),
    (F.col("preco_melhor_oferta_compra") / 100).cast(DecimalType(11,2)).alias("vl_melhor_oferta_compra"),
    (F.col("preco_melhor_oferta_venda") / 100).cast(DecimalType(11,2)).alias("vl_melhor_oferta_venda"),
    
    # Campos Inteiros
    F.col("numero_negocios_efetuados").cast(IntegerType()).alias("qt_negocios_efetuados"),
    F.col("quantidade_total_titulos").cast(IntegerType()).alias("qt_total_titulos"),
    
    # Volume com precisão 16,2 (mantido conforme especificação)
    (F.col("volume_total_titulos") / 100).cast(DecimalType(16,2)).alias("vl_total_titulos"),
    
    # Demais campos
    (F.col("preco_exercicio_opcoes") / 100).cast(DecimalType(11,2)).alias("vl_exercicio_opcoes"),
    F.col("indicador_correcao_precos").cast("string").alias("cd_indicador_correcao"),
    F.to_date(F.col("data_vencimento_opcoes"), "yyyyMMdd").alias("dt_vencimento_opcoes"),
    F.col("fator_cotacao_papel").cast("string").alias("cd_fator_cotacao"),
    (F.col("preco_exercicio_pontos") / 1000000).cast(DecimalType(7,6)).alias("vl_exercicio_pontos"),
    F.col("codigo_papel_sistema").cast("string").alias("cd_papel_sistema"),
    F.col("numero_distribuicao_papel").cast("string").alias("nr_distribuicao_papel"),
    F.col("nome_arquivo").cast("string").alias("nm_arquivo_origem"),
    
    # Adiciona timestamp de inserção
    F.current_timestamp().alias("ts_insercao")
)

In [6]:
# Salva o DataFrame transformado em formato Delta
df_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "false").save(delta_path_cotacao_historica)

print(f"[SUCESSO] DataFrame transformado gravado no Delta Lake em: {delta_path_cotacao_historica}")

[SUCESSO] DataFrame transformado gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica


## Stop Spark Session

In [7]:
# Encerra a SparkSession
spark.stop()